In [3]:
import torch
from torchdrug import datasets

dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True,
                            atom_feature="symbol")

Loading /Users/ca/molecule-datasets/250k_rndm_zinc_drugs_clean_3.csv:  50%|█████     | 249456/498911 [00:03<00:03, 70039.29it/s]
Constructing molecules from SMILES: 100%|██████████| 249455/249455 [05:56<00:00, 699.18it/s]


In [6]:
from pna import PNA
from torch_geometric.utils import degree

In [13]:
from gsn import GSN

In [7]:
deg = torch.zeros(10, dtype=torch.long)
for data in dataset:
    graph = data['graph']
    d = degree(graph.edge_list[:, 1], num_nodes=graph.num_node, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

PNA(input_dim=dataset.node_feature_dim,
                  hidden_dim=128, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  aggregators=['mean', 'min', 'max', 'std'],
                  scalers=['identity', 'amplification', 'attenuation'],
                  deg=deg, batch_norm=True)

In [14]:
from torchdrug import core, models, tasks

model = GSN(input_dim=dataset.node_feature_dim,
                  hidden_dim=128, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  batch_norm=True)
task = tasks.GCPNGeneration(model, dataset.atom_types, max_edge_unroll=12,
                            max_node=38, criterion="nll")

In [15]:
from torch import nn, optim
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     batch_size=128, log_interval=10)

solver.train(num_epoch=1)

11:51:05   Preprocess training set
11:51:05   {'batch_size': 128,
 'class': 'core.Engine',
 'gpus': None,
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 10,
          'atom_types': [6, 7, 8, 9, 15, 16, 17, 35, 53],
          'baseline_momentum': 0.9,
          'class': 'tasks.GCPNGeneration',
          'criterion': 'nll',
          'gamma': 0.9,
          'hidden_dim_mlp': 128,
          'max_edge_unroll': 12,
          'max_node': 38,
          'model': {'activation': 'relu',
                    'batch_norm': True,
                    'class': 'GSN',
                    'concat_hidden': False,
                    'edge_input_dim': 18,
                    'hidden_dim': 128,
                    'i

KeyboardInterrupt: 